In [1]:
import pandas
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names

2022-05-23 19:25:57.212685: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
saved_model_path = '/data/linghui/saved_model/deepfm-ml-1m/'

saved_data_path = '/data/linghui/ml-1m/processed_data/'

test_data_path = saved_data_path + 'test_data.csv'

#load data
test_data = pd.read_csv(test_data_path,index_col=0)

In [3]:
model_del_sf_path = saved_model_path + 'deepfm-del-sf/deepfm-ml-1m-del-sf.ckpt'
model_all_feat_path = saved_model_path + 'deepfm-all-feature/deepfm-ml-1m.ckpt' 

In [7]:
def get_feat_dict(data):
    if data == 'ml-1m':
        feat_dict = {'movie_id': 3706,
                 'user_id': 6040,
                 'gender': 2,
                 'age': 7,
                 'occupation': 21,
                 'zipcode': 3439}
    return feat_dict
dataset = 'ml-1m'
feat_dict = get_feat_dict(dataset)

In [8]:
sparse_features = ["movie_id", "user_id",
                   "gender", "age", "occupation", "zipcode"]

selected_feature = ["movie_id", "user_id"]

fixlen_feature_columns = [SparseFeat(feat, feat_dict[feat], embedding_dim=4)
								  for feat in sparse_features]
linear_feature_columns = fixlen_feature_columns 
dnn_feature_columns = fixlen_feature_columns 

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [10]:
check_path = model_all_feat_path
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary')
model.summary()
model.load_weights(check_path)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy',
              metrics=['AUC', 'Precision', 'Recall'])

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
gender (InputLayer)             [(None, 1)]          0                                            
__________________________________________________________________________________________________
age (InputLayer)                [(None, 1)]          0                                            
_______________________________________________________________________________________

In [12]:
test_input = {name: test_data[name].values for name in sparse_features}
label = test_data['rating'].values

In [13]:
batch_size = 256
test_auc = model.evaluate(test_input, label, batch_size=batch_size)[1]

2022-05-23 19:31:25.643440: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


1563/1563 [==============================] - 6s 4ms/step - loss: 0.4073 - auc: 0.9052 - precision: 0.8127 - recall: 0.8478


In [14]:
test_auc

0.9052456617355347

In [16]:
fixlen_feature_columns_del_sf = [SparseFeat(feat, feat_dict[feat], embedding_dim=4)
								  for feat in selected_feature]
linear_feature_columns_del_sf = fixlen_feature_columns_del_sf 
dnn_feature_columns_del_sf = fixlen_feature_columns_del_sf 

feature_names = get_feature_names(linear_feature_columns_del_sf + dnn_feature_columns_del_sf)

In [17]:
check_path = model_del_sf_path
model = DeepFM(linear_feature_columns_del_sf, dnn_feature_columns_del_sf, task='binary')
model.summary()
model.load_weights(check_path)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy',
              metrics=['AUC', 'Precision', 'Recall'])

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_id (InputLayer)           [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
sparse_emb_movie_id (Embedding) (None, 1, 4)         14824       movie_id[0][0]                   
__________________________________________________________________________________________________
sparse_emb_user_id (Embedding)  (None, 1, 4)         24160       user_id[0][0]                    
_______________________________________________________________________________________

In [18]:
batch_size = 256
test_auc = model.evaluate(test_input, label, batch_size=batch_size)[1]

/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:540: UserWarning: Input dict contained keys ['gender', 'age', 'occupation', 'zipcode'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


1563/1563 [==============================] - 5s 3ms/step - loss: 0.4104 - auc: 0.9018 - precision: 0.8087 - recall: 0.8444


In [ ]:
test_auc